## Pandas数据合并

导包

In [1]:
import numpy as np
import pandas as pd

- pd.concat
- pd.append
- pd.merge 

为方便讲解，我们首先定义一个生成DataFrame的函数：

In [2]:
def make_df(indexs, columns):    
    data = [[str(j)+str(i) for j in columns] for i in indexs]
    df = pd.DataFrame(data=data, index=indexs, columns=columns)
    return df

In [3]:
make_df([1, 2, 3, 4], list('ABCD'))

,A,B,C,D
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4


### 1. 使用pd.concat()级联

pandas使用pd.concat函数，与np.concatenate函数类似

#### 1)  简单级联

In [4]:
df1 = make_df([1, 2], ['A', 'B'])
df2 = make_df([3, 4], ['A', 'B'])
display(df1, df2)

,A,B
1,A1,B1
2,A2,B2


,A,B
3,A3,B3
4,A4,B4


In [5]:
# 默认上下合并，垂直合并
pd.concat([df1, df2])  

,A,B
1,A1,B1
2,A2,B2
3,A3,B3
4,A4,B4


In [6]:
# 左右合并，水平合并
pd.concat([df1, df2], axis=1)  

,A,B,A,B
1,A1,B1,NaN,NaN
2,A2,B2,NaN,NaN
3,NaN,NaN,A3,B3
4,NaN,NaN,A4,B4


- 忽略行索引 ignore_index 

In [7]:
# 忽略行索引：重置索引
pd.concat([df1, df2], ignore_index=True)

,A,B
0,A1,B1
1,A2,B2
2,A3,B3
3,A4,B4


- 使用多层索引 keys  

In [8]:
# 使用多层索引 keys
pd.concat([df1, df2], keys=['x', 'y'])
pd.concat([df1, df2], keys=['x', 'y'], axis=1)

x         y     
     A    B    A    B
1   A1   B1  NaN  NaN
2   A2   B2  NaN  NaN
3  NaN  NaN   A3   B3
4  NaN  NaN   A4   B4

#### 2) 不匹配级联

不匹配指的是级联的维度的索引不一致。例如纵向级联时列索引不一致，横向级联时行索引不一致

In [9]:
df3 = make_df([1, 2, 3, 4], list('ABCD'))
df4 = make_df([2, 3, 4, 5], list('BCDE'))
display(df3, df4)

,A,B,C,D
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4


,B,C,D,E
2,B2,C2,D2,E2
3,B3,C3,D3,E3
4,B4,C4,D4,E4
5,B5,C5,D5,E5


In [10]:
# 对应索引没有值，会自动用NaN填充
pd.concat([df3, df4])

,A,B,C,D,E
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
4,A4,B4,C4,D4,NaN
2,NaN,B2,C2,D2,E2
3,NaN,B3,C3,D3,E3
4,NaN,B4,C4,D4,E4
5,NaN,B5,C5,D5,E5


- 外连接：补NaN（默认模式）

In [11]:
# 外连接：类似并集，显示所有数据
pd.concat([df3, df4])
pd.concat([df3, df4], join='outer')

,A,B,C,D,E
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
4,A4,B4,C4,D4,NaN
2,NaN,B2,C2,D2,E2
3,NaN,B3,C3,D3,E3
4,NaN,B4,C4,D4,E4
5,NaN,B5,C5,D5,E5


- 内连接：只连接匹配的项

In [12]:
# 内连接： 类似交集， 只显示共同的部分
pd.concat([df3, df4], join='inner')

,B,C,D
1,B1,C1,D1
2,B2,C2,D2
3,B3,C3,D3
4,B4,C4,D4
2,B2,C2,D2
3,B3,C3,D3
4,B4,C4,D4
5,B5,C5,D5


#### 3) 使用append()函数添加

由于在后面级联的使用非常普遍，因此有一个函数append专门用于在后面添加

In [13]:
display(df3, df4)

,A,B,C,D
1,A1,B1,C1,D1
2,A2,B2,C2,D2
3,A3,B3,C3,D3
4,A4,B4,C4,D4


,B,C,D,E
2,B2,C2,D2,E2
3,B3,C3,D3,E3
4,B4,C4,D4,E4
5,B5,C5,D5,E5


In [14]:
df3.append(df4)

,A,B,C,D,E
1,A1,B1,C1,D1,NaN
2,A2,B2,C2,D2,NaN
3,A3,B3,C3,D3,NaN
4,A4,B4,C4,D4,NaN
2,NaN,B2,C2,D2,E2
3,NaN,B3,C3,D3,E3
4,NaN,B4,C4,D4,E4
5,NaN,B5,C5,D5,E5


### 2. 使用pd.merge()合并

- 类似MySQL中表和表直接的合并
- merge与concat的区别在于，merge需要依据某一共同的行或列来进行合并
- 使用pd.merge()合并时，会自动根据两者相同column名称的那一列，作为key来进行合并。
- 每一列元素的顺序不要求一致

####  1) 一对一合并

In [15]:
df1 = pd.DataFrame({
    'name': ['张三', '李四', '王五'],
    'id': [1, 2, 3],
    'age': [22, 33, 44]
})
df2 = pd.DataFrame({
    'id': [2, 3, 4],
    'sex': ['男', '女', '男'],
    'job': ['Saler', 'CEO', 'Programer']
})
display(df1, df2)

,name,id,age
0,张三,1,22
1,李四,2,33
2,王五,3,44


,id,sex,job
0,2,男,Saler
1,3,女,CEO
2,4,男,Programer


In [16]:
# 合并: 
# pd.merge(df1, df2)
df1.merge(df2)

,name,id,age,sex,job
0,李四,2,33,男,Saler
1,王五,3,44,女,CEO


#### 2) 多对一合并

In [17]:
df3 = pd.DataFrame({
    'name': ['张三', '李四', '王五'],
    'id': [1, 2, 2],
    'age': [22, 33, 44]
})
df4 = pd.DataFrame({
    'id': [2, 3, 4],
    'sex': ['男', '女', '男'],
    'job': ['Saler', 'CEO', 'Programer']
})
display(df3, df4)

,name,id,age
0,张三,1,22
1,李四,2,33
2,王五,2,44


,id,sex,job
0,2,男,Saler
1,3,女,CEO
2,4,男,Programer


In [18]:
df3.merge(df4)

,name,id,age,sex,job
0,李四,2,33,男,Saler
1,王五,2,44,男,Saler


#### 3) 多对多合并

In [19]:
df5 = pd.DataFrame({
    'name': ['张三', '李四', '王五'],
    'id': [1, 2, 2],
    'age': [22, 33, 44]
})
df6 = pd.DataFrame({
    'id': [2, 2, 4],
    'sex': ['男', '女', '男'],
    'job': ['Saler', 'CEO', 'Programer']
})
display(df5, df6)

,name,id,age
0,张三,1,22
1,李四,2,33
2,王五,2,44


,id,sex,job
0,2,男,Saler
1,2,女,CEO
2,4,男,Programer


In [20]:
df5.merge(df6)

,name,id,age,sex,job
0,李四,2,33,男,Saler
1,李四,2,33,女,CEO
2,王五,2,44,男,Saler
3,王五,2,44,女,CEO


#### 4) key的规范化

- 使用on=显式指定哪一列为key,当2个DataFrame有多列相同时使用

In [21]:
df1 = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['张三', '李四', '王五'],
    'age': [22, 33, 44]
})
df2 = pd.DataFrame({
    'id': [2, 3, 4],
    'name': ['王五', '女', '男'],
    'job': ['Saler', 'CEO', 'Programer']
})
display(df1, df2)

,id,name,age
0,1,张三,22
1,2,李四,33
2,3,王五,44


,id,name,job
0,2,王五,Saler
1,3,女,CEO
2,4,男,Programer


In [22]:
# 如果有多列名称相同，需要指定一列作为连接的字段
# df1.merge(df2, on='id')
df1.merge(df2, on='name')

,id_x,name,age,id_y,job
0,3,王五,44,2,Saler


- 使用left_on和right_on指定左右两边的列作为key，当左右两边的key都不想等时使用

In [23]:
df1 = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['张三', '李四', '王五'],
    'age': [22, 33, 44]
})
df2 = pd.DataFrame({
    'id2': [2, 3, 4],
    'sex': ['男', '女', '男'],
    'job': ['Saler', 'CEO', 'Programer']
})
display(df1, df2)

,id,name,age
0,1,张三,22
1,2,李四,33
2,3,王五,44


,id2,sex,job
0,2,男,Saler
1,3,女,CEO
2,4,男,Programer


In [24]:
# 如果没有相同的列名，则需要使用left_on，right_on来分别指定2个表中的不同列作为连接的字段
df1.merge(df2, left_on='id', right_on='id2')

,id,name,age,id2,sex,job
0,2,李四,33,2,男,Saler
1,3,王五,44,3,女,CEO


- 当左边的列和右边的index相同的时候,使用right_index=True

In [25]:
df1 = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['张三', '李四', '王五'],
    'age': [22, 33, 44]
})
df2 = pd.DataFrame({
    'id2': [2, 3, 4],
    'sex': ['男', '女', '男'],
    'job': ['Saler', 'CEO', 'Programer']
})
display(df1, df2)

,id,name,age
0,1,张三,22
1,2,李四,33
2,3,王五,44


,id2,sex,job
0,2,男,Saler
1,3,女,CEO
2,4,男,Programer


In [26]:
# 可以使用行索引作为连接的字段
df1.merge(df2, left_index=True, right_index=True)
df1.merge(df2, left_index=True, right_on='id2')

,id,name,age,id2,sex,job
0,3,王五,44,2,男,Saler


#### 5) 内合并与外合并

- 内合并：只保留两者都有的key（默认模式）

In [27]:
df1 = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['张三', '李四', '王五'],
    'age': [22, 33, 44]
})
df2 = pd.DataFrame({
    'id': [2, 3, 4],
    'sex': ['男', '女', '男'],
    'job': ['Saler', 'CEO', 'Programer']
})
display(df1, df2)

,id,name,age
0,1,张三,22
1,2,李四,33
2,3,王五,44


,id,sex,job
0,2,男,Saler
1,3,女,CEO
2,4,男,Programer


In [28]:
df1.merge(df2)  # 默认是内连接 inner join
df1.merge(df2, how='inner')

,id,name,age,sex,job
0,2,李四,33,男,Saler
1,3,王五,44,女,CEO


- 外合并 how='outer'：补NaN

In [29]:
# 外连接 ： 会显示2个表的所有数据
df1.merge(df2, how='outer')

,id,name,age,sex,job
0,1,张三,22.0,NaN,NaN
1,2,李四,33.0,男,Saler
2,3,王五,44.0,女,CEO
3,4,NaN,NaN,男,Programer


- 左合并、右合并：how='left'，how='right'，

In [30]:
# 左连接: 显示左边df1表的所有数据和右边表df2的公共数据
df1.merge(df2, how='left')

,id,name,age,sex,job
0,1,张三,22,NaN,NaN
1,2,李四,33,男,Saler
2,3,王五,44,女,CEO


In [31]:
# 右连接: 显示右边df2表的所有数据和左边表df1的公共数据
df1.merge(df2, how='right')

,id,name,age,sex,job
0,2,李四,33.0,男,Saler
1,3,王五,44.0,女,CEO
2,4,NaN,NaN,男,Programer


#### 6) 列冲突的解决

当列冲突时，即有多个列名称相同时，需要使用on=来指定哪一个列作为key，配合suffixes指定冲突列名

可以使用suffixes=自己指定后缀

In [32]:
df1 = pd.DataFrame({
    'id': [1, 2, 3],
    'name': ['张三', '李四', '王五'],
    'age': [22, 33, 44]
})
df2 = pd.DataFrame({
    'id': [2, 3, 4],
    'name': ['男', '女', '男'],
    'job': ['Saler', 'CEO', 'Programer']
})
display(df1, df2)

,id,name,age
0,1,张三,22
1,2,李四,33
2,3,王五,44


,id,name,job
0,2,男,Saler
1,3,女,CEO
2,4,男,Programer


In [33]:
df1.merge(df2, on='id', suffixes=['_df1', '_df2'])

,id,name_df1,age,name_df2,job
0,2,李四,33,男,Saler
1,3,王五,44,女,CEO


merge合并总结:

   - 合并有三种现象: 一对一, 多对一, 多对多.
   - 合并默认会找相同的列名进行合并, 如果有多个列名相同,用on来指定.
   - 如果没有列名相同,但是数据又相同,可以通过left_on, right_on来分别指定要合并的列.
   - 如果想和index合并, 使用left_index, right_index来指定.
   - 如果多个列相同,合并之后可以通过suffixes来区分.
   - 还可以通过how来控制合并的结果, 默认是内合并, 还有外合并outer, 左合并left, 右合并right.
